#  추천 시스템 - ETF 데이터 수집 및 전처리 


---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

## ETF 데이터 수집

- CSV 다운로드
- http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC020103010901

In [3]:
import pandas as pd
import numpy as np

etf_list = pd.read_csv('data/etf_list.csv', encoding='cp949')
etf_list.head()

,종목코드,종목명,상장일,분류체계,운용사,수익률(최근 1년),기초지수,추적오차,순자산총액,괴리율,변동성,복제방법,총보수,과세유형
0,466400,1Q 25-08 회사채(A+이상)액티브,2023/09/19,채권-회사채-단기,하나자산운용,4.52,KIS 2025-08만기형 크레딧 A+이상 지수(총수익),0.11,111916276404,0.03,매우낮음,실물(액티브),0.10,배당소득세(보유기간과세)
1,491610,1Q CD금리액티브(합성),2024/09/24,기타,하나자산운용,0.00,KIS 하나 CD금리 총수익지수,0.05,316206006696,0.02,매우낮음,합성(액티브),0.02,배당소득세(보유기간과세)
2,451060,1Q K200액티브,2023/01/31,주식-시장대표,하나자산운용,-3.66,코스피 200,0.77,99754348820,-0.01,높음,실물(액티브),0.18,배당소득세(보유기간과세)
3,463290,1Q 단기금융채액티브,2023/08/03,채권-혼합-단기,하나자산운용,4.01,MK 머니마켓 지수(총수익),0.05,252717462257,0.00,매우낮음,실물(액티브),0.08,배당소득세(보유기간과세)
4,479080,1Q 머니마켓액티브,2024/04/02,채권-혼합-단기,하나자산운용,0.00,KIS-하나 MMF 지수(총수익),0.06,308255065986,-0.01,매우낮음,실물(액티브),0.05,배당소득세(보유기간과세)


In [4]:
etf_list.tail()

,종목코드,종목명,상장일,분류체계,운용사,수익률(최근 1년),기초지수,추적오차,순자산총액,괴리율,변동성,복제방법,총보수,과세유형
925,429870,히어로즈 리츠이지스액티브,2022/05/24,부동산-리츠,키움투자자산운용,-4.51,iSelect 리츠 지수,3.07,3763127456,-0.98,낮음,실물(액티브),0.300,배당소득세(분리과세부동산ETF)
926,476450,히어로즈 머니마켓액티브,2024/02/29,채권-혼합-단기,키움투자자산운용,0.00,KIS-키움 MMF지수(총수익지수),0.06,327428123675,0.00,매우낮음,실물(액티브),0.050,배당소득세(보유기간과세)
927,460270,히어로즈 미국달러SOFR금리액티브(합성),2023/06/20,기타,키움투자자산운용,16.59,Solactive SOFR Daily Total Return Index,0.09,9885129694,-0.07,매우낮음,합성(액티브),0.050,배당소득세(보유기간과세)
928,459790,히어로즈 미국성장기업30액티브,2023/06/27,주식-시장대표,키움투자자산운용,44.70,MSCI USA Index(Price Return),6.44,13371801852,0.31,높음,실물(액티브),0.760,배당소득세(보유기간과세)
929,454780,히어로즈 종합채권(AA-이상)액티브,2023/04/11,채권-혼합-중기,키움투자자산운용,7.56,KIS 종합 채권시장지수(AA-이상)(총수익지수),0.24,338847308716,0.26,매우낮음,실물(액티브),0.025,배당소득세(보유기간과세)


In [5]:
etf_list.shape

(930, 14)

In [6]:
etf_list[etf_list['종목코드'] == '466400']

,종목코드,종목명,상장일,분류체계,운용사,수익률(최근 1년),기초지수,추적오차,순자산총액,괴리율,변동성,복제방법,총보수,과세유형
0,466400,1Q 25-08 회사채(A+이상)액티브,2023/09/19,채권-회사채-단기,하나자산운용,4.52,KIS 2025-08만기형 크레딧 A+이상 지수(총수익),0.11,111916276404,0.03,매우낮음,실물(액티브),0.1,배당소득세(보유기간과세)


In [7]:
strIsurCd = '45979'
etf_url = f"https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd={strIsurCd}"

etf_url

'https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979'

## Crawl4AI

- LLM, AI 에이전트, 데이터 파이프라인을 위한 **고성능 웹 크롤링** 솔루션 제공

- **오픈소스** 기반으로 실시간 성능과 유연한 구현이 가능

- https://github.com/unclecode/crawl4ai?tab=readme-ov-file

- 설치 방법:
    ```bash
    # 기본 설치 - 코어 라이브러리만 설치
    pip install crawl4ai

    # 초기 설정 - Playwright 브라우저 설치 및 환경 점검
    crawl4ai-setup

    # 진단 도구 - 시스템 호환성 확인
    crawl4ai-doctor
    ```

### 1) Jupyter 환경 설정

- `nest_asyncio`는 Jupyter에서 이미 실행 중인 이벤트 루프 위에 중첩된 이벤트 루프를 실행할 수 있게 해주는 패키지

- Jupyter Notebook 에서 asyncio 를 사용할 때, 이 코드를 실행하면 에러가 발생하지 않음 

In [8]:
# Jupyter Notebook 에서 asyncio 를 사용할 때, 이 코드를 실행하면 에러가 발생하지 않음 (MacOS, Linux 환경)
import nest_asyncio
nest_asyncio.apply()

In [9]:
# # Jupyter Notebook 에서 asyncio 를 사용할 때, 이 코드를 실행하면 에러가 발생하지 않음 (Windows용)
import asyncio
import sys

# 이벤트 루프 정책 변경 (Windows용)
if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
# 새 이벤트 루프 생성 및 설정
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

# nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply(loop)

### 2) **기본 크롤링** 

- `BrowserConfig`와 `CrawlerRunConfig` 클래스 기본값 설정을 사용

In [10]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig

async def main():
    """ 
    크롤링 실행 함수
    """
    browser_config = BrowserConfig()  # 브라우저 설정
    run_config = CrawlerRunConfig()   # 크롤러 실행 설정

    async with AsyncWebCrawler(config=browser_config) as crawler:   # 크롤러 객체 생성
        result = await crawler.arun(
            url=etf_url,
            config=run_config   # 크롤러 실행 설정
        )
        
    return result


# 크롤링 실행 및 결과 출력
# result = asyncio.run(main())  # loop.run_until_complete() 사용
result = loop.run_until_complete(main())  # 새 이벤트 루프에서 실행
print(result.markdown)  # 크롤링 결과 (정제된 markdown 출력)

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 2.67s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 2.69s 

# 키움 KIWOOM 미국성장기업30액티브증권상장지수투자신탁[주식]
  * [상품개요(ETF)](https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979 "상품개요\(ETF\)")


## 상품 개요
목록 한글명 |  KIWOOM 미국성장기업30액티브  | 영문명 |  KIWOOM US Growth 30 Active   
---|---|---|---  
표준코드 |  KR7459790002  | 종목코드 |  459790   
상장일 |  2023-06-27  | 펀드형태 |  수익증권형   
기초지수명 |  MSCI USA Index(Price Return)  | 추적배수 |  일반 (1)   
자산운용사 |  키움투자자산운용(주)  | 지정참가회사(AP) |  키움증권, 삼성증권, DB금융투자   
총보수(%) |  0.76  | 회계기간 |  매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연도 12월 31일까지   
과세유형 |  배당소득세(보유기간과세)  | 분배금 지급일 |  1월, 4월, 7월, 10월 마지막 영업일 및 회계기간 종료일(다만, 회계기간 종료일이 영업일이 아닌 경우 그 직전 영업일)   
홈페이지 |  <http://www.kosef.co.kr>  
## 유형 정보
목록 기초 시장 |  (해외)   
(북미)   
(미국)  | 기초 자산 |  (주식)   
(시장대표) / (-)   
(-) / (-)   
---|---|---|---  
## 상품 설명
목록 기본 정보 |  ```
1. 이 투자신탁은 해외 주식을 주된 투자대상으로 하며, MSCI에서 산출 및 발표하는 “MSCI USA Index(Price Return)(원화환산)”를 비교지수로 하여 1좌당 순자산가치의 변동률이 비교지수의 변동률을 초과하도록 운용하는 것을 목적으로 합니다.  

2. 이 투자신탁은 액티브상장지수펀드로서 비교지

### 3) **CrawlResult 응답 객체** 

- 크롤링된 URL, HTML, 성공 여부 등 기본 정보 포함

- **cleaned_html**와 **markdown** 필드로 정제된 데이터 접근 가능

- **extracted_content**를 통해 구조화된 형태의 추출 데이터 확인

In [11]:
result.model_dump()

{'url': 'https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979',
 'html': '<!DOCTYPE html><html lang="ko"><head>\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\n<meta http-equiv="content-language" content="kr">\n<meta http-equiv="content-style-type" content="text/css">\n<meta http-equiv="pragma" content="no-cache">\n<meta http-equiv="cache-control" content="no-cache">\n<meta http-equiv="imagetoolbar" content="no">\n<meta name="copyright" content="(c)2012">\n<meta name="description" content="대한민국 대표 기업공시채널 KIND">\n<meta name="keywords" content="대한민국 대표 기업공시채널 KIND">\n<title>대한민국 대표 기업공시채널 KIND</title>\n<link rel="stylesheet" type="text/css" href="../js/jquery/themes-base/jquery-ui.min.css" media="all">\n<link rel="stylesheet" type="text/css" href="../css/default_new.css" media="all">\n<link rel="stylesheet" type="text/css" href="../css/common.css?version=20240531" media="a

In [12]:
# 크롤링 성공 여부 확인
result.success

True

In [13]:
# HTML 접근
print(result.html[:100])

<!DOCTYPE html><html lang="ko"><head>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta ht


In [14]:
# 정제된 HTML 출력
print(result.cleaned_html[:100])

<html>
<head>
<title>대한민국 대표 기업공시채널 KIND</title>
<!--[if lt IE 9]>
<script type="text/javascript" sr


In [15]:
# 마크다운 변환 결과 (정제된)
print(result.markdown)

# 키움 KIWOOM 미국성장기업30액티브증권상장지수투자신탁[주식]
  * [상품개요(ETF)](https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979 "상품개요\(ETF\)")


## 상품 개요
목록 한글명 |  KIWOOM 미국성장기업30액티브  | 영문명 |  KIWOOM US Growth 30 Active   
---|---|---|---  
표준코드 |  KR7459790002  | 종목코드 |  459790   
상장일 |  2023-06-27  | 펀드형태 |  수익증권형   
기초지수명 |  MSCI USA Index(Price Return)  | 추적배수 |  일반 (1)   
자산운용사 |  키움투자자산운용(주)  | 지정참가회사(AP) |  키움증권, 삼성증권, DB금융투자   
총보수(%) |  0.76  | 회계기간 |  매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연도 12월 31일까지   
과세유형 |  배당소득세(보유기간과세)  | 분배금 지급일 |  1월, 4월, 7월, 10월 마지막 영업일 및 회계기간 종료일(다만, 회계기간 종료일이 영업일이 아닌 경우 그 직전 영업일)   
홈페이지 |  <http://www.kosef.co.kr>  
## 유형 정보
목록 기초 시장 |  (해외)   
(북미)   
(미국)  | 기초 자산 |  (주식)   
(시장대표) / (-)   
(-) / (-)   
---|---|---|---  
## 상품 설명
목록 기본 정보 |  ```
1. 이 투자신탁은 해외 주식을 주된 투자대상으로 하며, MSCI에서 산출 및 발표하는 “MSCI USA Index(Price Return)(원화환산)”를 비교지수로 하여 1좌당 순자산가치의 변동률이 비교지수의 변동률을 초과하도록 운용하는 것을 목적으로 합니다.  

2. 이 투자신탁은 액티브상장지수펀드로서 비교지

In [16]:
# 구조화된 데이터 (Optional)
if result.extracted_content:
    data = json.loads(result.extracted_content)
    print(data)

### 4) **BrowserConfig 설정** 

- **BrowserConfig**로 브라우저 타입, 헤드리스 모드, 뷰포트 크기 등 **기본 설정** 관리

- **프록시 설정**과 **디버깅 옵션**을 통한 고급 크롤링 환경 구성

- **AsyncWebCrawler** 클래스와 연동하여 비동기 크롤링 실행

In [17]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig

# 기본 설정
base_config = BrowserConfig(
    browser_type="chromium",  # 브라우저 엔진 선택
    viewport_width=1080,     # 뷰포트 크기
    viewport_height=600,
    text_mode=True,          # 텍스트 모드 (이미지 비활성화)
    use_persistent_context=True,  # 세션을 유지
    headless=True,           # 헤드리스 모드 실행
)

# 디버깅용 설정
debug_config = base_config.clone(
    headless=False,  # 헤드리스 모드 비활성화
    verbose=True     # 디버깅용 로그 출력
) 

# 프록시 설정 (필요 시 적절한 프록시 서버 구성)
proxy_config = {
    "server": "http://proxy.example.com:8080",
    "username": "user",
    "password": "pass"
}

proxy_config = base_config.clone(
    proxy_config=proxy_config,  
)

# 크롤러 실행 설정
run_config = CrawlerRunConfig()   

async def extract_by_url(url , browser_config: BrowserConfig, run_config: CrawlerRunConfig):
    """ 
    크롤링 실행 함수
    """

    async with AsyncWebCrawler(config=browser_config) as crawler:   # 크롤러 객체 생성
        result = await crawler.arun(
            url=url,
            config=run_config   # 크롤러 실행 설정
        )
        
    return result


# 크롤링 실행 및 결과 출력
#result = asyncio.run(extract_by_url(etf_url, base_config, run_config))
result = loop.run_until_complete(extract_by_url(etf_url, base_config, run_config))
print(result.success)  # 크롤링 성공 여부
print(result.markdown)  # 크롤링 결과 (정제된 markdown 출력)

[BROWSER]. ℹ Starting browser with args: 
C:\Users\kaydash\AppData\Local\ms-playwright\chromium-1187\chrome-win\chrome.exe --remote-debugging-port=9222 
--user-data-dir=C:\Users\kaydash\AppData\Local\Temp\browser-profile-sv6gupoy --headless=new --disable-gpu 
--disable-gpu-compositing --disable-software-rasterizer --no-sandbox --disable-dev-shm-usage --no-first-run 
--no-default-browser-check --disable-infobars --window-position=0,0 --ignore-certificate-errors 
--ignore-certificate-errors-spki-list --disable-blink-features=AutomationControlled --window-position=400,0 
--disable-renderer-backgrounding --disable-ipc-flooding-protection --force-color-profile=srgb --mute-audio 
--disable-background-timer-throttling --blink-settings=imagesEnabled=false --disable-remote-fonts --disable-images 
--disable-javascript 

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.78s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.80s 

True
# 키움 KIWOOM 미국성장기업30액티브증권상장지수투자신탁[주식]
  * [상품개요(ETF)](https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979 "상품개요\(ETF\)")


## 상품 개요
목록 한글명 |  KIWOOM 미국성장기업30액티브  | 영문명 |  KIWOOM US Growth 30 Active   
---|---|---|---  
표준코드 |  KR7459790002  | 종목코드 |  459790   
상장일 |  2023-06-27  | 펀드형태 |  수익증권형   
기초지수명 |  MSCI USA Index(Price Return)  | 추적배수 |  일반 (1)   
자산운용사 |  키움투자자산운용(주)  | 지정참가회사(AP) |  키움증권, 삼성증권, DB금융투자   
총보수(%) |  0.76  | 회계기간 |  매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연도 12월 31일까지   
과세유형 |  배당소득세(보유기간과세)  | 분배금 지급일 |  1월, 4월, 7월, 10월 마지막 영업일 및 회계기간 종료일(다만, 회계기간 종료일이 영업일이 아닌 경우 그 직전 영업일)   
홈페이지 |  <http://www.kosef.co.kr>  
## 유형 정보
목록 기초 시장 |  (해외)   
(북미)   
(미국)  | 기초 자산 |  (주식)   
(시장대표) / (-)   
(-) / (-)   
---|---|---|---  
## 상품 설명
목록 기본 정보 |  ```
1. 이 투자신탁은 해외 주식을 주된 투자대상으로 하며, MSCI에서 산출 및 발표하는 “MSCI USA Index(Price Return)(원화환산)”를 비교지수로 하여 1좌당 순자산가치의 변동률이 비교지수의 변동률을 초과하도록 운용하는 것을 목적으로 합니다.  

2. 이 투자신탁은 액티브상장지수펀드로

### 5) **CrawlerRunConfig 설정** 

- 단어 수 제한, 추출 전략, 캐시 설정 등 **크롤링 옵션** 관리

- **js_code**와 **wait_for** 옵션으로 동적 콘텐츠 처리 가능

- **screenshot**과 **rate_limiting** 기능으로 크롤링 과정 제어

- **기본 구조**:

    ```python
    class CrawlerRunConfig:
        def __init__(
            word_count_threshold=200,      # 컨텐츠 최소 단어 수
            extraction_strategy=None,      # 데이터 추출 전략
            cache_mode=None,               # 캐시 설정
            js_code=None,                  # 실행할 JS 코드
            wait_for=None,                 # 대기 조건
            screenshot=False,              # 스크린샷 캡처
            enable_rate_limiting=False     # 속도 제한 활성화
        )
    ```

`(1) 기본 추출 설정`

In [18]:
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

run_config = CrawlerRunConfig(
    cache_mode=CacheMode.ENABLED,   # 캐시 활성화
    word_count_threshold=200,       # 컨텐츠 블록의 최소 단어 수 기준 설정 (짧은 문단이나 항목이 많은 사이트는 기준값을 낮춰야 함)
)

# 크롤링 실행 및 결과 출력
# result = asyncio.run(extract_by_url(etf_url, base_config, run_config))
result = loop.run_until_complete(extract_by_url(etf_url, base_config, run_config))
print(result.success)  # 크롤링 성공 여부
print(result.markdown)  # 크롤링 결과 (정제된 markdown 출력)

[BROWSER]. ℹ Starting browser with args: 
C:\Users\kaydash\AppData\Local\ms-playwright\chromium-1187\chrome-win\chrome.exe --remote-debugging-port=9222 
--user-data-dir=C:\Users\kaydash\AppData\Local\Temp\browser-profile-f4qa4d40 --headless=new --disable-gpu 
--disable-gpu-compositing --disable-software-rasterizer --no-sandbox --disable-dev-shm-usage --no-first-run 
--no-default-browser-check --disable-infobars --window-position=0,0 --ignore-certificate-errors 
--ignore-certificate-errors-spki-list --disable-blink-features=AutomationControlled --window-position=400,0 
--disable-renderer-backgrounding --disable-ipc-flooding-protection --force-color-profile=srgb --mute-audio 
--disable-background-timer-throttling --blink-settings=imagesEnabled=false --disable-remote-fonts --disable-images 
--disable-javascript 

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.07s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.07s 

True
# 키움 KIWOOM 미국성장기업30액티브증권상장지수투자신탁[주식]
  * [상품개요(ETF)](https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979 "상품개요\(ETF\)")


## 상품 개요
목록 한글명 |  KIWOOM 미국성장기업30액티브  | 영문명 |  KIWOOM US Growth 30 Active   
---|---|---|---  
표준코드 |  KR7459790002  | 종목코드 |  459790   
상장일 |  2023-06-27  | 펀드형태 |  수익증권형   
기초지수명 |  MSCI USA Index(Price Return)  | 추적배수 |  일반 (1)   
자산운용사 |  키움투자자산운용(주)  | 지정참가회사(AP) |  키움증권, 삼성증권, DB금융투자   
총보수(%) |  0.76  | 회계기간 |  매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연도 12월 31일까지   
과세유형 |  배당소득세(보유기간과세)  | 분배금 지급일 |  1월, 4월, 7월, 10월 마지막 영업일 및 회계기간 종료일(다만, 회계기간 종료일이 영업일이 아닌 경우 그 직전 영업일)   
홈페이지 |  <http://www.kosef.co.kr>  
## 유형 정보
목록 기초 시장 |  (해외)   
(북미)   
(미국)  | 기초 자산 |  (주식)   
(시장대표) / (-)   
(-) / (-)   
---|---|---|---  
## 상품 설명
목록 기본 정보 |  ```
1. 이 투자신탁은 해외 주식을 주된 투자대상으로 하며, MSCI에서 산출 및 발표하는 “MSCI USA Index(Price Return)(원화환산)”를 비교지수로 하여 1좌당 순자산가치의 변동률이 비교지수의 변동률을 초과하도록 운용하는 것을 목적으로 합니다.  

2. 이 투자신탁은 액티브상장지수펀드로

`(2) 추출 전략`

- 참조 문서: https://docs.crawl4ai.com/core/content-selection/

- 대상 페이지: https://news.ycombinator.com/newest

- robots.txt: https://www.ycombinator.com/robots.txt

In [19]:
news_url = "https://news.ycombinator.com/newest"
#news_url = "https://kind.krx.co.kr/disclosure/disclosurebystocktype.do?method=searchDisclosureByStockTypeEtf"
news_url

'https://news.ycombinator.com/newest'

In [20]:
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

run_config = CrawlerRunConfig(
    css_selector="main.content",         # 크롤링 대상 컨텐츠 선택자
    word_count_threshold=10,             # 컨텐츠 블록의 최소 단어 수 기준 설정
    excluded_tags=["nav", "footer"],     # 제외할 태그 설정
    exclude_external_links=True,         # 외부 링크 제외 설정
    exclude_social_media_links=True,     # 소셜 미디어 링크 제외 설정
    exclude_domains=["ads.com", "spammytrackers.net"],   # 제외할 도메인 설정
    exclude_external_images=True,         # 외부 이미지 제외 설정
    cache_mode=CacheMode.BYPASS           # 캐시 비활성화
)

# 크롤링 실행 및 결과 출력
# result = asyncio.run(extract_by_url(news_url, base_config, run_config))
result = loop.run_until_complete(extract_by_url(etf_url, base_config, run_config))
print(result.success)  # 크롤링 성공 여부
print(result.markdown)  # 크롤링 결과 (정제된 markdown 출력)

[BROWSER]. ℹ Starting browser with args: 
C:\Users\kaydash\AppData\Local\ms-playwright\chromium-1187\chrome-win\chrome.exe --remote-debugging-port=9222 
--user-data-dir=C:\Users\kaydash\AppData\Local\Temp\browser-profile-drhp6c61 --headless=new --disable-gpu 
--disable-gpu-compositing --disable-software-rasterizer --no-sandbox --disable-dev-shm-usage --no-first-run 
--no-default-browser-check --disable-infobars --window-position=0,0 --ignore-certificate-errors 
--ignore-certificate-errors-spki-list --disable-blink-features=AutomationControlled --window-position=400,0 
--disable-renderer-backgrounding --disable-ipc-flooding-protection --force-color-profile=srgb --mute-audio 
--disable-background-timer-throttling --blink-settings=imagesEnabled=false --disable-remote-fonts --disable-images 
--disable-javascript 

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.78s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.00s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.78s 

True




In [ ]:
# CSS 기반 추출 전략
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy

news_schema = {
    "name": "HN Stories",
    "baseSelector": "tr.athing",
    "fields": [
        {
            "name": "title",
            "selector": "td.title span.titleline a:first-child",
            "type": "text"
        },
        {
            "name": "url",
            "selector": "td.title span.titleline a:first-child",
            "type": "attribute", 
            "attribute": "href"
        },
        {
            "name": "rank",
            "selector": "td.title span.rank",
            "type": "text",
            "transform": lambda x: int(x.strip("."))
        }
    ]
}

extraction_config = CrawlerRunConfig(
    extraction_strategy=JsonCssExtractionStrategy(news_schema),
    cache_mode=CacheMode.BYPASS  # 항상 최신 데이터 가져오기
)


# 크롤링 실행 및 결과 출력
# result = asyncio.run(extract_by_url(news_url, base_config, extraction_config))
result = loop.run_until_complete(extract_by_url(etf_url, base_config, run_config))

# 구조화된 데이터 (Optional)
if result.extracted_content:
    data = json.loads(result.extracted_content)
    print(data)

[BROWSER]. ℹ Starting browser with args: 
C:\Users\kaydash\AppData\Local\ms-playwright\chromium-1187\chrome-win\chrome.exe --remote-debugging-port=9222 
--user-data-dir=C:\Users\kaydash\AppData\Local\Temp\browser-profile-459ihr__ --headless=new --disable-gpu 
--disable-gpu-compositing --disable-software-rasterizer --no-sandbox --disable-dev-shm-usage --no-first-run 
--no-default-browser-check --disable-infobars --window-position=0,0 --ignore-certificate-errors 
--ignore-certificate-errors-spki-list --disable-blink-features=AutomationControlled --window-position=400,0 
--disable-renderer-backgrounding --disable-ipc-flooding-protection --force-color-profile=srgb --mute-audio 
--disable-background-timer-throttling --blink-settings=imagesEnabled=false --disable-remote-fonts --disable-images 
--disable-javascript 

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.90s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.00s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.90s 

In [22]:
if result.extracted_content:
    data = json.loads(result.extracted_content)
    print(data)

In [23]:
print(f"Type: {type(result.extracted_content)}")
print(f"Value: {result.extracted_content}")

if result.extracted_content:
    data = json.loads(result.extracted_content)
    print(data[0])
else:
    print("추출된 콘텐츠가 없습니다. extraction_strategy를 사용했는지 확인하세요.")

Type: <class 'NoneType'>
Value: None
추출된 콘텐츠가 없습니다. extraction_strategy를 사용했는지 확인하세요.


In [24]:
# 네이버 뉴스

news_url = "https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews"
news_url

'https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews'

In [25]:
# CSS 기반 추출 전략
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy

news_schema = {
    "name": "NewsArticles",
    "baseSelector": "div.newsct",
    "fields": [
        {
            "name": "title",
            "selector": "div.media_end_head div.media_end_head_title h2.media_end_head_headline span",
            "type": "text"
        },
    ]
}

extraction_config = CrawlerRunConfig(
    extraction_strategy=JsonCssExtractionStrategy(news_schema),
    process_iframes=True,
    remove_overlay_elements=True
)


news_url = "https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews"

# 크롤링 실행 및 결과 출력
# result = asyncio.run(extract_by_url(news_url, base_config, extraction_config))
result = loop.run_until_complete(extract_by_url(etf_url, base_config, extraction_config))

# 구조화된 데이터 (Optional)
if result.extracted_content:
    data = json.loads(result.extracted_content)
    print(data)

[BROWSER]. ℹ Starting browser with args: 
C:\Users\kaydash\AppData\Local\ms-playwright\chromium-1187\chrome-win\chrome.exe --remote-debugging-port=9222 
--user-data-dir=C:\Users\kaydash\AppData\Local\Temp\browser-profile-uz8edf9k --headless=new --disable-gpu 
--disable-gpu-compositing --disable-software-rasterizer --no-sandbox --disable-dev-shm-usage --no-first-run 
--no-default-browser-check --disable-infobars --window-position=0,0 --ignore-certificate-errors 
--ignore-certificate-errors-spki-list --disable-blink-features=AutomationControlled --window-position=400,0 
--disable-renderer-backgrounding --disable-ipc-flooding-protection --force-color-profile=srgb --mute-audio 
--disable-background-timer-throttling --blink-settings=imagesEnabled=false --disable-remote-fonts --disable-images 
--disable-javascript 

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 1.65s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.01s 

[EXTRACT]. ■ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=45979         |
✓ | ⏱: 1.68s 

[]


In [26]:
import os
import json
import asyncio
from pydantic import BaseModel, Field
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
from crawl4ai.async_configs import LLMConfig 
from crawl4ai.extraction_strategy import LLMExtractionStrategy

# 데이터 모델 정의
class NaverNewsArticle(BaseModel):
    title: str = Field(description="기사 제목")
    published_date: str = Field(description="발행일")
    author: str = Field(description="기자 이름") 
    content: str = Field(description="기사 본문")


async def extract_naver_news(news_url):
    # LLM 추출 전략 설정
    strategy = LLMExtractionStrategy(
        llm_config=LLMConfig(
            provider="openai/gpt-4.1-mini",
            api_token=os.getenv("OPENAI_API_KEY"),
            ),
        schema=NaverNewsArticle.model_json_schema(),
        extraction_type="schema",
        instruction="""
        네이버 뉴스 기사에서 다음 정보를 추출하세요:
        - title: 기사 제목
        - published_date: 발행일시 
        - author: 기자 이름
        - content: 기사 본문
        """
    )

    config = CrawlerRunConfig(
        exclude_external_links=True,
        extraction_strategy=strategy,
        cache_mode=CacheMode.BYPASS
    )

    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(url=news_url, config=config)

        print(result.extracted_content)
        
        if result.extracted_content:
            article = json.loads(result.extracted_content)
            return article
        return None

# 실행
news_url = "https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews"
# article = asyncio.run(extract_naver_news(news_url)) # loop.run_until_complete() 사용
article = loop.run_until_complete(extract_naver_news(news_url))  #  윈도우 기준


print(article)

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews                  |
✓ | ⏱: 0.71s 

[SCRAPE].. ◆ https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews                  |
✓ | ⏱: 0.11s 

[EXTRACT]. ■ https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews                  |
✓ | ⏱: 19.32s 

[COMPLETE] ● https://n.news.naver.com/article/654/0000103393?cds=news_media_pc&type=breakingnews                  |
✓ | ⏱: 20.15s 

[
    {
        "title": "미제…안전 확인 안 되는 한국인 80여 명",
        "published_date": "2025-10-14",
        "author": "강원도민일보",
        "content": "14일 외교부에 따르면 캄보디아에 입국했다가 연락 두절 또는 감금됐다는 신고가 들어온 한국인 숫자는 올해 1~8월 330명, 지난해 220명으로 파악됐다. 이 가운데 올해 인원 260여 명, 지난해 210명은 ‘종결’ 처리됐다. 최근 캄보디아에서 한국인 상대 취업 사기와 납치, 감금·고문·살해 등 강력범죄가 잇따르고 있으며, 충북에서는 올해 실종 신고 대상자 중 3명의 행방이 아직 확인되지 않고 있다. 주한미군 소속 한국인 노동자들이 미국 연방정부 셧다운 여파로 10월 급여를 받지 못하는 상황도 발생했다. 대통령실은 캄보디아에 구금된 한국인 63명에 대해 송환 특별기 투입 등 다각적인 방안을 협의 중이라고 밝혔다. 캄보디아 한인회 김대윤 부회장은 한국인들이 통장과 개인정보를 팔러 와 중국 조직에 납치·감금·사망하는 일이 일상화됐다고 전했다. 한국 정부의 대응이 안일하다는 지적도 나오고 있다.",
        "error": false
    },
    {
        "title": "[속보] 전국 곳곳 대설 특보…강원내륙·산지 최대 30㎝ 폭설",
        "published_date": "2025.01.27. 오전 8:57",
        "author": "이채윤 기자",
        "content": "임시공휴일이자 월요일인 27일 전국에 눈·비가 쏟아졌다가 잦아들기를 반복하겠다. 길게는 설날까지 이어질 전망이다. 오전 7시 30분 현재 동해안 일부를 제외한 전국에 눈 또는 비가 내리고 있다. 북쪽에서 찬 공기가 남하해 들어오면서 비가 오는 지역은 남해안 일부로 줄고 대부분 지역에선 눈이 내리고 있다. 제주 한라산엔 오전 7시 기준 최근 24시간 내 10㎝ 넘는 눈이 내려 쌓였다. 강원 철원

In [27]:
pprint(article[0]['content'])

('14일 외교부에 따르면 캄보디아에 입국했다가 연락 두절 또는 감금됐다는 신고가 들어온 한국인 숫자는 올해 1~8월 330명, 지난해 '
 '220명으로 파악됐다. 이 가운데 올해 인원 260여 명, 지난해 210명은 ‘종결’ 처리됐다. 최근 캄보디아에서 한국인 상대 취업 '
 '사기와 납치, 감금·고문·살해 등 강력범죄가 잇따르고 있으며, 충북에서는 올해 실종 신고 대상자 중 3명의 행방이 아직 확인되지 않고 '
 '있다. 주한미군 소속 한국인 노동자들이 미국 연방정부 셧다운 여파로 10월 급여를 받지 못하는 상황도 발생했다. 대통령실은 캄보디아에 '
 '구금된 한국인 63명에 대해 송환 특별기 투입 등 다각적인 방안을 협의 중이라고 밝혔다. 캄보디아 한인회 김대윤 부회장은 한국인들이 '
 '통장과 개인정보를 팔러 와 중국 조직에 납치·감금·사망하는 일이 일상화됐다고 전했다. 한국 정부의 대응이 안일하다는 지적도 나오고 있다.')


In [29]:
# ================================================================================
# [실습 완성] ETF 상세정보 수집 - 수정된 버전
# ================================================================================

import pandas as pd
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

# 1. ETF 종목코드 5개 선택
sample_etf_codes = etf_list['종목코드'].head(5).tolist()
print(f"수집 대상 ETF 종목코드: {sample_etf_codes}")

# 2. ETF 상세정보 추출 함수 (수정됨)
async def extract_etf_detail_fixed(etf_code):
    """
    ETF 상세 페이지에서 정보를 추출하는 함수 (수정 버전)
    """
    # 종목코드를 5자리 문자열로 변환
    str_isur_cd = str(etf_code).zfill(5)
    etf_url = f"https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd={str_isur_cd}"
    
    print(f"\n크롤링 중: {etf_code} - {etf_url}")
    
    # 크롤링 설정
    browser_config = BrowserConfig(
        browser_type="chromium",
        headless=True,
        text_mode=False,
        viewport_width=1080,
        viewport_height=600
    )
    
    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        word_count_threshold=10
    )
    
    # 크롤링 실행
    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(url=etf_url, config=run_config)
    
    if not result.success:
        print(f"❌ 크롤링 실패: {etf_code}")
        return None
    
    # pandas.read_html()로 테이블 추출
    try:
        tables = pd.read_html(result.html)
        print(f"  테이블 개수: {len(tables)}")
        
        # ETF 정보를 저장할 딕셔너리 (기본값)
        etf_info = {
            '종목코드': etf_code,
            '한글명': None,
            '영문명': None,
            '표준코드': None,
            '상장일': None,
            '펀드형태': None,
            '기초지수명': None,
            '추적배수': None,
            '자산운용사': None,
            '지정참가회사(AP)': None,
            '총보수(%)': None,
            '회계기간': None,
            '과세유형': None,
            '분배금 지급일': None,
            '홈페이지': None,
            '기초 시장': None,
            '기초 자산': None,
            '기본 정보': None,
            '투자유의사항': None
        }
        
        # 올바른 테이블 찾기: 2열 형식이고 '한글명' 항목이 있는 테이블
        target_table = None
        for table in tables:
            # 2열 형식 확인
            if table.shape[1] == 2:
                # 첫 번째 열에 '한글명'이 있는지 확인
                if '한글명' in table.iloc[:, 0].values:
                    target_table = table
                    print(f"  ✅ ETF 정보 테이블 발견! Shape: {table.shape}")
                    break
        
        if target_table is None:
            print(f"  ⚠️ ETF 정보 테이블을 찾을 수 없습니다")
            return etf_info
        
        # 테이블에서 데이터 추출 (정확한 키 매칭)
        for idx, row in target_table.iterrows():
            key = str(row.iloc[0]).strip()
            value = str(row.iloc[1]).strip()
            
            # 정확한 키 매칭 (existing data structure 기반)
            if key == '한글명':
                etf_info['한글명'] = value
            elif key == '영문명':
                etf_info['영문명'] = value
            elif key == '표준코드':
                etf_info['표준코드'] = value
            elif key == '종목코드':
                pass  # 이미 설정됨
            elif key == '상장일':
                etf_info['상장일'] = value
            elif key == '펀드형태':
                etf_info['펀드형태'] = value
            elif key == '기초지수명':
                etf_info['기초지수명'] = value
            elif key == '추적배수':
                etf_info['추적배수'] = value
            elif key == '자산운용사':
                etf_info['자산운용사'] = value
            elif key == '지정참가회사(AP)':
                etf_info['지정참가회사(AP)'] = value
            elif key == '총보수(%)':
                etf_info['총보수(%)'] = value
            elif key == '회계기간':
                etf_info['회계기간'] = value
            elif key == '과세유형':
                etf_info['과세유형'] = value
            elif key == '분배금 지급일':
                etf_info['분배금 지급일'] = value
            elif key == '홈페이지':
                etf_info['홈페이지'] = value
            elif key == '기초 시장':
                etf_info['기초 시장'] = value
            elif key == '기초 자산':
                etf_info['기초 자산'] = value
            elif key == '기본 정보':
                etf_info['기본 정보'] = value
            elif key == '투자유의사항':
                etf_info['투자유의사항'] = value
        
        # 추출 성공 확인
        filled_count = sum(1 for v in etf_info.values() if v is not None)
        print(f"  📊 추출된 필드: {filled_count}/19")
        
        return etf_info
        
    except Exception as e:
        print(f"  ❌ 테이블 파싱 오류 ({etf_code}): {e}")
        return None


# 3. 5개 ETF에 대해 순차 크롤링
async def collect_all_etfs_fixed(etf_codes):
    """여러 ETF 정보를 순차적으로 수집"""
    results = []
    
    for code in etf_codes:
        etf_data = await extract_etf_detail_fixed(code)
        if etf_data:
            results.append(etf_data)
        # 서버 부하 방지
        await asyncio.sleep(1)
    
    return results


# 4. 크롤링 실행
print("\n" + "="*70)
print("ETF 상세정보 수집 시작")
print("="*70)

etf_details_fixed = loop.run_until_complete(collect_all_etfs_fixed(sample_etf_codes))

# 5. 결과 확인
print("\n" + "="*70)
print(f"✅ 수집 완료: {len(etf_details_fixed)}개 ETF")
print("="*70)

if etf_details_fixed:
    print("\n첫 번째 ETF 정보:")
    for key, value in etf_details_fixed[0].items():
        print(f"  {key}: {value}")

# 6. DataFrame으로 변환
df_etf_details_fixed = pd.DataFrame(etf_details_fixed)
print("\n수집된 데이터 미리보기:")
print(df_etf_details_fixed)

# 7. CSV 파일로 저장
output_path = 'data/etf_details_fixed.csv'
df_etf_details_fixed.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 CSV 저장 완료: {output_path}")

수집 대상 ETF 종목코드: ['466400', '491610', '451060', '463290', '479080']

ETF 상세정보 수집 시작

크롤링 중: 466400 - https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400


[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 13.51s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 13.52s 

...

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 3.42s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 3.44s 

C:\Users\kaydash\AppData\Local\Temp\ipykernel_33596\2214038668.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(result.html)


  테이블 개수: 3
  ⚠️ ETF 정보 테이블을 찾을 수 없습니다

✅ 수집 완료: 3개 ETF

첫 번째 ETF 정보:
  종목코드: 491610
  한글명: None
  영문명: None
  표준코드: None
  상장일: None
  펀드형태: None
  기초지수명: None
  추적배수: None
  자산운용사: None
  지정참가회사(AP): None
  총보수(%): None
  회계기간: None
  과세유형: None
  분배금 지급일: None
  홈페이지: None
  기초 시장: None
  기초 자산: None
  기본 정보: None
  투자유의사항: None

수집된 데이터 미리보기:
     종목코드   한글명   영문명  표준코드   상장일  펀드형태 기초지수명  추적배수 자산운용사 지정참가회사(AP) 총보수(%)  \
0  491610  None  None  None  None  None  None  None  None       None   None   
1  463290  None  None  None  None  None  None  None  None       None   None   
2  479080  None  None  None  None  None  None  None  None       None   None   

   회계기간  과세유형 분배금 지급일  홈페이지 기초 시장 기초 자산 기본 정보 투자유의사항  
0  None  None    None  None  None  None  None   None  
1  None  None    None  None  None  None  None   None  
2  None  None    None  None  None  None  None   None  

💾 CSV 저장 완료: data/etf_details_fixed.csv


---

## [실습] **ETF 상세정보 수집**

- 다음 ETF 상세 페이지에서 다음 항목을 수집하는 함수를 작성합니다. 
- 대상 필드:
    - '한글명', '영문명', '종목코드', '상장일', '펀드형태', '기초지수명', '추적배수', '자산운용사',
    - '지정참가회사(AP)', '총보수(%)', '회계기간', '과세유형', '분배금 지급일', '홈페이지',
    - '기초 시장', '기초 자산', '기본 정보', '투자유의사항'

- 모두 5개의 ETF 종목에 대한 정보를 수집하여 csv 파일로 저장합니다. 

- **Hint**:
    - crawl4ai 사용하여 각 ETF 상세 페이지의 HTML 소스코드를 추출 (종목코드: 5자리 숫자에 유의)
    - pandas.read_html() 함수를 사용하여 HTML 소스코드에서 테이블 데이터를 추출 

In [30]:
# ================================================================================
# [실습] ETF 상세정보 수집 - 완성 코드
# ================================================================================

import pandas as pd
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

# 1. ETF 종목코드 5개 선택 (etf_list에서 샘플링)
sample_etf_codes = etf_list['종목코드'].head(5).tolist()
print(f"수집 대상 ETF 종목코드: {sample_etf_codes}")

# 2. ETF 상세정보 추출 함수
async def extract_etf_detail(etf_code):
    """
    ETF 상세 페이지에서 정보를 추출하는 함수
    
    Args:
        etf_code: 5자리 ETF 종목코드 (예: 466400)
    
    Returns:
        dict: 추출된 ETF 상세정보
    """
    # 종목코드를 5자리 문자열로 변환 (앞자리가 0인 경우 대비)
    str_isur_cd = str(etf_code).zfill(5)
    etf_url = f"https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd={str_isur_cd}"

    print(f"\n크롤링 중: {etf_code} - {etf_url}")

    # 크롤링 설정
    browser_config = BrowserConfig(
        browser_type="chromium",
        headless=True,
        text_mode=False,  # 테이블 추출을 위해 False
        viewport_width=1080,
        viewport_height=600
    )

    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,  # 항상 최신 데이터
        word_count_threshold=10
    )

    # 크롤링 실행
    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(url=etf_url, config=run_config)

    if not result.success:
        print(f"크롤링 실패: {etf_code}")
        return None

    # pandas.read_html()로 테이블 추출
    try:
        tables = pd.read_html(result.html)
        print(f"테이블 개수: {len(tables)}")

        # 추출된 데이터를 저장할 딕셔너리
        etf_info = {
            '종목코드': etf_code,
            '한글명': None,
            '영문명': None,
            '상장일': None,
            '펀드형태': None,
            '기초지수명': None,
            '추적배수': None,
            '자산운용사': None,
            '지정참가회사(AP)': None,
            '총보수(%)': None,
            '회계기간': None,
            '과세유형': None,
            '분배금 지급일': None,
            '홈페이지': None,
            '기초 시장': None,
            '기초 자산': None,
            '기본 정보': None,
            '투자유의사항': None
        }

        # 테이블에서 필드 매핑 (테이블 구조에 따라 조정 필요)
        # 일반적으로 첫 번째 또는 두 번째 테이블에 기본 정보가 있음
        for table in tables:
            # 테이블이 2열 형식 (항목명, 값)인 경우
            if table.shape[1] >= 2:
                for idx, row in table.iterrows():
                    key = str(row[0]).strip()
                    value = str(row[1]).strip()

                    # 필드 매핑
                    if '한글명' in key or '종목명' in key:
                        etf_info['한글명'] = value
                    elif '영문명' in key:
                        etf_info['영문명'] = value
                    elif '상장일' in key:
                        etf_info['상장일'] = value
                    elif '펀드형태' in key or '형태' in key:
                        etf_info['펀드형태'] = value
                    elif '기초지수' in key:
                        etf_info['기초지수명'] = value
                    elif '추적배수' in key or '레버리지' in key:
                        etf_info['추적배수'] = value
                    elif '자산운용사' in key or '운용사' in key:
                        etf_info['자산운용사'] = value
                    elif '지정참가회사' in key or 'AP' in key:
                        etf_info['지정참가회사(AP)'] = value
                    elif '총보수' in key:
                        etf_info['총보수(%)'] = value
                    elif '회계기간' in key:
                        etf_info['회계기간'] = value
                    elif '과세유형' in key:
                        etf_info['과세유형'] = value
                    elif '분배금' in key:
                        etf_info['분배금 지급일'] = value
                    elif '홈페이지' in key:
                        etf_info['홈페이지'] = value
                    elif '기초 시장' in key or '시장' in key:
                        etf_info['기초 시장'] = value
                    elif '기초 자산' in key or '자산' in key:
                        etf_info['기초 자산'] = value
                    elif '기본 정보' in key:
                        etf_info['기본 정보'] = value
                    elif '투자유의사항' in key or '유의사항' in key:
                        etf_info['투자유의사항'] = value

        return etf_info

    except Exception as e:
        print(f"테이블 파싱 오류 ({etf_code}): {e}")
        return None


# 3. 5개 ETF에 대해 순차 크롤링 (Windows 환경)
async def collect_all_etfs(etf_codes):
    """여러 ETF 정보를 순차적으로 수집"""
    results = []

    for code in etf_codes:
        etf_data = await extract_etf_detail(code)
        if etf_data:
            results.append(etf_data)
        # 서버 부하 방지를 위한 딜레이
        await asyncio.sleep(1)

    return results


# 4. 크롤링 실행 (Windows - loop.run_until_complete 사용)
etf_details = loop.run_until_complete(collect_all_etfs(sample_etf_codes))

# 5. 결과 확인
print(f"\n수집된 ETF 개수: {len(etf_details)}")
if etf_details:
    print("\n첫 번째 ETF 정보:")
    print(etf_details[0])

# 6. DataFrame으로 변환
df_etf_details = pd.DataFrame(etf_details)
print("\n수집된 데이터:")
print(df_etf_details)

# 7. CSV 파일로 저장
output_path = 'data/etf_details.csv'
df_etf_details.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료: {output_path}")


수집 대상 ETF 종목코드: ['466400', '491610', '451060', '463290', '479080']

크롤링 중: 466400 - https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400


[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 3.41s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=466400        |
✓ | ⏱: 3.43s 

...

[FETCH]... ↓ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 2.75s 

[SCRAPE].. ◆ https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://kind.krx.co.kr/disclosure/etfisudetail.do?method=searchEtfIsuSummary&strIsurCd=479080        |
✓ | ⏱: 2.76s 

C:\Users\kaydash\AppData\Local\Temp\ipykernel_33596\3256852361.py:54: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(result.html)


테이블 파싱 오류 (479080): Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

수집된 ETF 개수: 3

첫 번째 ETF 정보:
{'종목코드': '466400', '한글명': 'nan', '영문명': None, '상장일': 'nan', '펀드형태': None, '기초지수명': 'nan', '추적배수': None, '자산운용사': 'nan', '지정참가회사(AP)': None, '총보수(%)': 'nan', '회계기간': None, '과세유형': 'nan', '분배금 지급일': None, '홈페이지': 'nan', '기초 시장': '-', '기초 자산': None, '기본 정보': 'nan', '투자유의사항': 'nan'}

수집된 데이터:
     종목코드  한글명   영문명  상장일  펀드형태 기초지수명  추적배수 자산운용사 지정참가회사(AP) 총보수(%)  회계기간  \
0  466400  nan  None  nan  None   nan  None   nan       None    nan  None   
1  491610  nan  None  nan  None   nan  None   nan       None    nan  None   
2  463290  nan  None  nan  None   nan  None   nan       None    nan  None   

  과세유형 분배금 지급일 홈페이지 기초 시장 기초 자산 기본 정보 투자유의사항  
0  nan    None  nan     -  None   nan    nan  
1  nan    None  nan     -  None   nan    nan  
2  nan    None  nan     -  None   nan    nan  

✅ CSV 저장 완료: data/etf_details.csv
